## 1. Persiapan

In [1]:
# import library yang dibutuhkan
import sqlite3
import pandas as pd

In [2]:
#upload file
from google.colab import files
uploaded = files.upload()

Saving northwind.db to northwind.db


In [3]:
# cek versi dari sqlite kita
sqlite3.sqlite_version

'3.37.2'

In [4]:
# membuat koneksi ke database
db = sqlite3.connect('northwind.db')

In [5]:
# buat variabel cursor dari koneksi ke database kita
cursor = db.cursor()

In [6]:
# query di bawah akan menampilkan tabel - tabel yang ada di database kita
query = """
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
"""

In [7]:
# menjalankan query dan menampilkan hasil
cursor.execute(query)
print(cursor.fetchall())

[('Suppliers',), ('Shippers',), ('Employees',), ('Customers',), ('Orders',), ('Categories',), ('Products',), ('OrderDetails',)]


## 2. JOIN

In [8]:
query = """
SELECT * FROM Customers a
limit 5
"""
pd.read_sql_query(query,db)

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67


In [9]:
query = """
SELECT * FROM orders b
limit 5
"""
pd.read_sql_query(query,db)

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2014-07-04 08:00:00.000,2014-08-01 00:00:00.000,2014-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,2014-07-05 04:00:00.000,2014-08-16 00:00:00.000,2014-07-10 00:00:00.000,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,2014-07-08 15:00:00.000,2014-08-05 00:00:00.000,2014-07-12 00:00:00.000,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,2014-07-08 14:00:00.000,2014-08-05 00:00:00.000,2014-07-15 00:00:00.000,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,2014-07-09 01:00:00.000,2014-08-06 00:00:00.000,2014-07-11 00:00:00.000,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium


INNER JOIN

In [ ]:
#gabungkan company name, ordedate, orderid dan freight menggunakan alias
query = """
SELECT
a.CompanyName,
date(b.OrderDate) as date_order,
b.OrderID,
b.freight

FROM Customers a
INNER JOIN Orders b
ON a.CustomerID = b.CustomerID

ORDER BY 1
"""
pd.read_sql_query(query,db)

,CompanyName,date_order,OrderID,Freight
0,Alfreds Futterkiste,2015-08-25,10643,29.46
1,Alfreds Futterkiste,2015-10-03,10692,61.02
2,Alfreds Futterkiste,2015-10-13,10702,23.94
3,Alfreds Futterkiste,2016-01-15,10835,69.53
4,Alfreds Futterkiste,2016-03-16,10952,40.42
...,...,...,...,...
825,Wolski Zajazd,2015-12-23,10792,23.79
826,Wolski Zajazd,2016-02-04,10870,12.04
827,Wolski Zajazd,2016-02-25,10906,26.29
828,Wolski Zajazd,2016-04-03,10998,20.31


In [ ]:
#cari total order dan freight berdasarkan company name dan bulan
query = """
SELECT
Customers.CompanyName,
date(Orders.OrderDate,'start of month') as date_order,
count(distinct Orders.OrderID) as total_order,
sum(orders.freight) as freight
FROM Customers
INNER JOIN Orders ON Customers.CustomerID = Orders.CustomerID
group by 1,2
ORDER BY 1,2
"""
pd.read_sql_query(query,db)

,CompanyName,date_order,total_order,freight
0,Alfreds Futterkiste,2015-08-01,1,29.46
1,Alfreds Futterkiste,2015-10-01,2,84.96
2,Alfreds Futterkiste,2016-01-01,1,69.53
3,Alfreds Futterkiste,2016-03-01,1,40.42
4,Alfreds Futterkiste,2016-04-01,1,1.21
...,...,...,...,...
631,Wolski Zajazd,2014-12-01,1,3.94
632,Wolski Zajazd,2015-07-01,1,80.65
633,Wolski Zajazd,2015-12-01,1,23.79
634,Wolski Zajazd,2016-02-01,2,38.33


LEFT JOIN

In [ ]:
#join product name dan order id dengan left join
query = """
SELECT
upper(productname),
count(distinct b.orderid)
FROM products a
LEFT JOIN Orderdetails b
on a.productid = b.productid
group by 1
"""
pd.read_sql(query, db)

,upper(productname),count(distinct b.orderid)
0,ALICE MUTTON,37
1,ANISEED SYRUP,12
2,BOSTON CRAB MEAT,41
3,CAMEMBERT PIERROT,51
4,CARNARVON TIGERS,27
...,...,...
72,UNCLE BOB'S ORGANIC DRIED PEARS,29
73,VALKOINEN SUKLAA,10
74,VEGIE-SPREAD,17
75,WIMMERS GUTE SEMMELKNöDEL,30


RIGHT JOIN

In [ ]:
#join product name dan order id dengan left join namun ditukar tablenya untuk menghasilkan right join
query = """
SELECT
upper(productname),
count(distinct b.orderid)
FROM Orderdetails b
LEFT JOIN products a
on a.productid = b.productid
group by 1
"""
pd.read_sql(query, db)

,upper(productname),count(distinct b.orderid)
0,ALICE MUTTON,37
1,ANISEED SYRUP,12
2,BOSTON CRAB MEAT,41
3,CAMEMBERT PIERROT,51
4,CARNARVON TIGERS,27
...,...,...
72,UNCLE BOB'S ORGANIC DRIED PEARS,29
73,VALKOINEN SUKLAA,10
74,VEGIE-SPREAD,17
75,WIMMERS GUTE SEMMELKNöDEL,30


UNION

In [ ]:
# menggabungkan companyname dari customers dan suppliers
query = """
SELECT companyname from suppliers
union
SELECT companyname from customers
"""
pd.read_sql(query, db)

,CompanyName
0,Alfreds Futterkiste
1,Ana Trujillo Emparedados y helados
2,Antonio Moreno Taquería
3,Around the Horn
4,Aux joyeux ecclésiastiques
...,...
115,Wellington Importadora
116,White Clover Markets
117,Wilman Kala
118,Wolski Zajazd


INTERSECT

In [ ]:
#mencari tau apakah semua employee pernah menjual pada tanggal tertentu
query = """
select employeeid from employees
intersect
select employeeid from orders
where date(OrderDate,'start of month') = '2014-08-01'
"""
pd.read_sql(query, db)

,EmployeeID
0,1
1,2
2,3
3,4
4,6
5,7
6,8


In [ ]:
#mencari tau apakah siapa employee tidak pernah menjual pada tanggal tertentu
query = """
select employeeid from employees
except
select employeeid from orders
where date(OrderDate,'start of month') = '2014-08-01'
"""
pd.read_sql(query, db)

,EmployeeID
0,5
1,9


# Mini Case

Selamat ! Kamu baru saja diterima bekerja sebagai seorang Jr. Data Analyst di perusahaan Nortwind Traders!. Hari ini adalah hari pertama kamu bekerja, dan manager kamu ingin kamu secara perlahan mengetahui seluk beluk dari perusahaan tempat kamu bekerja. Kamu diminta untuk mengerjakan beberapa tugas berikut menggunakan bantuan SQL yang sudah kamu pelajari sebelumnya.
Tugas kamu adalah membuat query untuk menjawab beberapa tugas yang akan diberikan di bawah ini, kemudian kamu dapat memberikan komentar sesuai dengan hasil query kamu.
Selamat bekerja!!

1. Saya ingin mendapatkan total order, rata-rata freight dan detail karyawan yang menjual (nama depan, nama belakang, tittle, kota)


2. Berapa total penjualan (unitprice * quantity) dari  setiap kategori product?